In [1]:
from gymnasium import Env
from gymnasium.spaces import Box
from typing import Any, SupportsFloat
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdt
import random

from grid import Microgrid
from agents import MLAgent

from stable_baselines3 import A2C

import warnings
warnings.filterwarnings("ignore")

2023-10-21 19:57:02.827345: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_parquet('./data/power_price.parquet')
df = df.dropna()
days = df['time'].dt.date.unique()

# dataset = []

# for day in days:
#     part = df[df['time'].dt.date == day]
#     if len(part) == 288:
#         dataset.append({'power' : part['power'].to_numpy(),
#                         'price' : part['price'].to_numpy()})

env = Microgrid()
env.reset(df)

(array([0. , 0.2, 1. , 0. , 0. ]), {'ts_start': 20444})

In [3]:
model = A2C('MlpPolicy', env, verbose=0)
m = model.learn(total_timesteps=28800)

In [4]:
def simpleAlgo(state):
    t, batt, price, sol, hydro = state
    return np.array([1, 1 - price])


In [5]:
df = pd.read_csv('data/HOLLISTR_1_N101.csv')
df = df[['INTERVALSTARTTIME_GMT', 'MW', 'LMP_TYPE']]
df = df[df['LMP_TYPE'] == 'LMP']
df.drop(columns=['LMP_TYPE'], inplace=True)
df.reset_index(inplace=True, drop=True)
df.columns = ['dt', 'MW']
df['dt'] = pd.to_datetime(df['dt'].str.slice(0, -6).str.replace('T', ' '))
# df['date'] = df['dt'].dt.date
# df['time'] =  df['dt'].dt.hour * 12 + df['dt'].dt.minute // 5
# df.drop(columns=['dt'], inplace=True)
df

agent = MLAgent()
agent.load_history(df.set_index('dt'))

In [7]:
episodes = 10
# env.verbose = True
for ep in range(episodes):
    seed = random.randint(0, 1e9)

    state, info = env.reset(seed=seed)
    done = False
    score_model = 0
    while not done:
        action, state = model.predict(state)
        state, reward, done, trunc, info = env.step(action)
        score_model += reward
    
    state, info = env.reset(seed=seed)
    done = False
    score_algo = 0
    while not done:
        action = simpleAlgo(state)
        state, reward, done, trunc, info = env.step(action)
        score_algo += reward
    
    state, info = env.reset(seed=seed)
    agent.reset(info)
    done = False
    score_agent = 0
    while not done:
        action = agent.predict(state)
        state, reward, done, trunc, info = env.step(action)
        score_agent += reward
    print(f'Episode {ep+1}: score_model {score_model},\
score_algo {score_algo}, score_agent {score_agent}')

Episode 1: score_model 424.88848333105983,score_algo 700.980605766899, score_agent 1099.0934246865659
Episode 2: score_model 1913.202676152816,score_algo 2188.0866044637382, score_agent 1297.1747390992284
Episode 3: score_model 221.34098451930024,score_algo 227.2647449577429, score_agent 208.11968586874164
Episode 4: score_model 232.58584120165807,score_algo 220.47344731763394, score_agent 206.1153906844099
Episode 5: score_model 330.60736192749164,score_algo 379.6650372723298, score_agent 281.9526614976177
Episode 6: score_model 264.0479534012527,score_algo 272.0606317820366, score_agent 232.0510351731367
Episode 7: score_model 516.626588810546,score_algo 574.0449548231339, score_agent 364.0244083960932
Episode 8: score_model 357.8198711547278,score_algo 375.6695126813228, score_agent 336.4225625102358
Episode 9: score_model 448.846098634485,score_algo 497.50165843958246, score_agent 384.33616757927894
Episode 10: score_model 318.9115307818083,score_algo 375.72527296431576, score_agen